# Exploring Ebay Car Sales

## Description

The goal of this project is to practice Python data cleaning and analysis techniques using the Pandas library.

### Data
    
The dataset used in this project contains car sales data from eBay Kleinanzeigen, a German classifieds section of eBay. The working dataset is a sample of 50,000 entries that was "dirtied" to better resemble original raw data that one may scrape from a website. The full, cleaned version of the dataset is available on [Kaggle.](https://www.kaggle.com/orgesleka/used-cars-database/data)

A data dictionary was provided with the original dataset:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

## Data Import

We'll begin the analysis by first importing the data as a pandas dataframe. The data is held in a file called 'autos.csv'. The `.info()` and `.head()` methods will be used to provide an initial look at the dataset.

In [150]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding='Latin-1')

autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The dataset size is as expected with 50,000 rows and 20 columns. A few of the columns are of the `int64` dtype, indicating only numeric data in its values. There are some columns that are not `int64` type, but likely should be. Columns such as `odometer`, `price` contain string values. Looking at the first rows, we see that the '$' and 'km' are included to indicate the units used.

One issue that stands out right away are the columns that don't contain the expected 50,000 values:
`vehicleType`, `gearbox`, `model`, `fuelType`, and `notRepairedDamage`. We cannot say at this point why these values are missing, but a look at the first few rows gives an idea of what type of data should be expected. One note to make is that none of these columns are missing more than 10,000 rows.

## Data Cleaning

The first step for any analysis is to examine and clean the data. Any analysis done without cleaning may lead to skewed or invalid results. This section covers several important steps for cleaning this data and preparing it for further analysis.

### Column Names

One noticeable issue is some of the column names are lengthy or aren't immediately clear what data they contain. In this next step we'll rename the columns to be more readable and clear using snakecase.



In [151]:
new_columns = {'dateCrawled':'date_crawled',
               'offerType':'offer_type',
               'vehicleType':'vehicle_type', 'yearOfRegistration':'registration_year', 
               'gearbox':'gear_box', 'powerPS':'power_PS',
               'monthOfRegistration':'registration_month', 'fuelType':'fuel_type',
               'notRepairedDamage':'unrepaired_damage', 'dateCreated':'ad_created', 'nrOfPictures':'num_pictures',
               'postalCode':'postal_code', 'lastSeen':'last_seen'}

autos.rename(columns=new_columns, inplace=True)

autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Changing the column names makes it easier to understand and reference their content. Using the snakecase format allows for quick readability. The next steps of our analysis won't require examining the data again just to remember which columns contained what information, as it can be inferred with our cleaned column names.

### Unnecessary Data

Next we'll investigate the data itself. Firstly we should look for any unnecessary columns or those that can have data converted to a single type. For example a string column type that actually contains numeric data. The `dataframe.describe()` method shows descriptive statistics for the entire dataframe.

In [152]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gear_box,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


A few conclusions can be made using this output:
- 'seller' column is primarily one value: 'privat'
- 'offer_type' column is primarily one value: 'Angebot'
- 'price' and 'odometer' columns are string types but actually contain numeric information
- 'registration_year' may have invalid values, showing a minimum of '1000' and a maximum of '9999' which are not valid years

Let's first remove the 'seller' and 'offer_type' columns. Since these are a single value for all but one entry, they will not provide any useful information for analysis.

In [153]:
autos.drop(columns = ['seller','offer_type'], inplace=True)

### Column Data Types

Next we'll clean convert the 'odometer' and 'price' columns to numeric data types. To do so we have to remove any string characters in the values. Examining the series values using the `value_counts()` method will reveal what characters are used.

In [154]:
autos['odometer'].value_counts().head()

150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
Name: odometer, dtype: int64

In [155]:
autos['price'].value_counts().head()

$0        1421
$500       781
$1,500     734
$2,500     643
$1,000     639
Name: price, dtype: int64

For 'odometer', the unit indicator 'km' and commas need to be removed. For 'price', the dollar sign and commas need to be removed. Since the units within the values are being removed, the names of the columns need to be changed to reflect the units.

In [156]:
autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')

autos = autos.astype({'odometer':'int64','price':'int64'})
autos.rename(columns = {'odometer':'odometer_km','price':'price_dollars'}, inplace=True)

### Outliers and Invalid Entries

Now that the 'price' and 'odometer' columns are numeric, we can examine that data more appropriately. In particular we'll search for any outliers or unusual values in the data. Methods like `describe()`, `value_counts()`, and `sort_values()` are useful for examining the data.

In [157]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

The odometer data does not appear to have any significant outliers. However, the value counts show that only exact values at certain intervals are selected. It's likely that odometer data is selected as a category in the listing rather than the exact reading.

In [158]:
autos['price_dollars'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollars, dtype: float64

In [159]:
autos['price_dollars'].sort_values(ascending=False)

39705    99999999
42221    27322222
39377    12345678
47598    12345678
27371    12345678
2897     11111111
24384    11111111
11137    10000000
47634     3890000
7814      1300000
22947     1234566
43049      999999
514        999999
37585      999990
36818      350000
14715      345000
34723      299000
35923      295000
12682      265000
47337      259000
38299      250000
37840      220000
40918      198000
43668      197000
28090      194000
20351      190000
17140      180000
11433      175000
32840      169999
18509      169000
           ...   
21171           0
24900           0
1220            0
31264           0
24871           0
2470            0
21256           0
29503           0
37021           0
21253           0
47861           0
29482           0
31336           0
35819           0
27020           0
4266            0
2481            0
15208           0
33620           0
12819           0
31332           0
35821           0
8438            0
43925           0
38832     

Price data does have some peculiarities. The maximum price is \$99,999,999, and other prices on the high end have likely invalid values like '12345678'. Looking at the sorted values shows a distinct cutoff at \$350,000, jumping to \$999,990 and higher. Although some of these values may be possible, it's unlikely that they are valid. Therefore we'll remove these top values setting a cutoff of \$350,000.

In [160]:
autos = autos[autos['price_dollars'] <= 350000]
autos.describe(include='all')

,date_crawled,name,price_dollars,abtest,vehicle_type,registration_year,gear_box,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,49986,49986,49986.000000,49986,44894,49986.000000,47310,49986.000000,47233,49986.000000,49986.000000,45509,49986,40163,49986,49986.0,49986.000000,49986
unique,48200,38743,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,39472
top,2016-03-23 19:38:20,Ford_Fiesta,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,NaN,25750,12854,NaN,36985,NaN,4024,NaN,NaN,30100,10684,35225,1946,NaN,NaN,8
mean,NaN,NaN,5721.525167,NaN,NaN,2005.075721,NaN,116.341196,NaN,125736.506222,5.723723,NaN,NaN,NaN,NaN,0.0,50812.804225,NaN
std,NaN,NaN,8983.617820,NaN,NaN,105.727161,NaN,209.218012,NaN,40038.133399,3.711839,NaN,NaN,NaN,NaN,0.0,25777.404967,NaN
min,NaN,NaN,0.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,1100.000000,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,2950.000000,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49571.000000,NaN
75%,NaN,NaN,7200.000000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71522.000000,NaN


Next, let's examine the columns containing date information. The 'date_crawled', 'ad_created', and 'last_seen' columns are meant to be date values, but instead are held as strings.

Looking at some of the values in the series will show what format the date strings are in.

In [161]:
autos[['date_crawled','ad_created','last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


The three columns are each in the 'yyyy-mm-dd hh:mm:ss' format. Since we are only interested in the date itself, we can extract just the first 10 characters. This will leave the columns in string format, but allows for examining the value counts based on day, not including time data.

In [162]:
autos['date_crawled'] = autos['date_crawled'].str[:10]
autos['ad_created'] = autos['ad_created'].str[:10]
autos['last_seen'] = autos['last_seen'].str[:10]
autos[['date_crawled','ad_created','last_seen']].describe()

,date_crawled,ad_created,last_seen
count,49986,49986,49986
unique,34,76,34
top,2016-04-03,2016-04-03,2016-04-06
freq,1934,1946,11046


The other two columns related to date are 'registration_year' and 'registration_month'. We'll examine those columns to ensure that data is valid and useful.

Using the 'describe()' method shows anomalies with both columns. 'Registration_year' has a minimum of 1000, and a max of 9999. Both of these years are obviously invalid. The 'registration_month' column has a minimum of 0. Intuitively this should be a '1' and may need to be investigated.

To clean 'registration_year' data, we'll examine the values that fall outside of the range 1900 - 2016. This interval includes all plausible registration years given the earliest appearances of cars, and the last date the data was collected.

In [163]:
autos[['registration_year','registration_month']].describe()

,registration_year,registration_month
count,49986.000000,49986.000000
mean,2005.075721,5.723723
std,105.727161,3.711839
min,1000.000000,0.000000
25%,1999.000000,3.000000
50%,2003.000000,6.000000
75%,2008.000000,9.000000
max,9999.000000,12.000000


In [164]:
autos['registration_year'][~autos['registration_year'].between(1900,2016)].value_counts()

2017    1452
2018     491
9999       4
5000       4
2019       3
9000       2
1800       2
6200       1
4500       1
8888       1
4800       1
2800       1
1001       1
1000       1
1111       1
1500       1
9996       1
5911       1
4100       1
Name: registration_year, dtype: int64

The values within 'registration_year' show a notable amount were listed as registered in 2017 and 2018. Although the data was collected in 2016, there may be some valid reason for a registration date in later years, such as listing a renewal date instead of the original date. Further investigation into how 'registration_year' is collected may be necessary. For the purpose of this analysis, we'll include 2017 and 2018 entries and remove all others, most of which are clearly invalid although include 3 listings showing 2019 as the registration year.

In [165]:
autos = autos[autos['registration_year'].between(1900,2018)]
autos['registration_year'].describe()

count    49959.000000
mean      2003.367862
std          7.688527
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2018.000000
Name: registration_year, dtype: float64

## Brand Analysis

The cleaned data is now ready for analysis. One of the aspects to investigate is the differences between brands in price and mileage.

First, we'll identify brands that are of interest in this analysis. A good start would be retrieving the top 10 most common brands in the data.

In [166]:
top10_brands = autos['brand'].value_counts()[:10].index
print(top10_brands)

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat'],
      dtype='object')


Next, we'll loop through each brand and calculate the mean price and distance. The output will be converted into a series, then the series input into a dataframe.

In [167]:
brand_price_dict = {}
for brand in top10_brands:
    brand_price_dict[brand] = autos['price_dollars'][autos['brand']==brand].mean()

brand_distance_dict = {}
for brand in top10_brands:
    brand_distance_dict[brand] = autos['odometer_km'][autos['brand']==brand].mean()

    
price_series = pd.Series(brand_price_dict)
distance_series = pd.Series(brand_distance_dict)

brand_df = pd.DataFrame(price_series, columns=['mean_price'])
brand_df['mean_km'] = distance_series
print(brand_df.sort_values(by='mean_price', ascending=False))

                mean_price        mean_km
audi           8965.560355  129643.941163
mercedes_benz  8380.637920  130933.206510
bmw            8028.474479  132540.998710
volkswagen     5159.401629  129006.461279
seat           4223.654255  122186.170213
ford           3626.542997  124153.005464
peugeot        3010.868819  127352.335165
opel           2842.824629  129361.370717
fiat           2697.677123  117012.241775
renault        2351.301997  128223.793677


In [168]:
print(brand_df.sort_values(by='mean_km', ascending=False))

                mean_price        mean_km
bmw            8028.474479  132540.998710
mercedes_benz  8380.637920  130933.206510
audi           8965.560355  129643.941163
opel           2842.824629  129361.370717
volkswagen     5159.401629  129006.461279
renault        2351.301997  128223.793677
peugeot        3010.868819  127352.335165
ford           3626.542997  124153.005464
seat           4223.654255  122186.170213
fiat           2697.677123  117012.241775


Looking at price, it's clear that luxury brands like Audi, Mercedes, and BMW are priced higher than all other brands. Volkswagen, Seat, and Ford make up a mid-range grouping in price, while Peugeot, Opel, Fiat, and Renault fall lower. Odometer measurements are not as variable. Except for Fiat at 117,012km, the remaining brands fall within a grouping of 11,000km between 122,000km and 132,000km.

## Additional Analysis

The purpose of this project was to demonstrate data cleaning techniques using the Pandas library. Much more analysis can be done with the cleaned data to investigate a number of interesting aspects. Some follow-up analyses may include:

- Price / distance relationship
- Price / damage relationship
- Vehicle type by brand